In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from IPython.display import SVG, Image
from livelossplot.inputs.tf_keras import PlotLossesCallback
import tensorflow as tf
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
# calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
from sklearn.metrics import f1_score
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.6.0


In [2]:
img_size = 48
batch_size = 64
datagen_train = ImageDataGenerator(horizontal_flip=True)
train_generator = datagen_train.flow_from_directory("../../content",
                                                    target_size=(img_size,img_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)
train_generator


Found 0 images belonging to 3 classes.


In [3]:
# Initialising the CNN
model = Sequential()
# 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(40, 30,9)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# 2nd Convolution layer
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# 3rd Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# 4th Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# Flattening
model.add(Flatten())
# Fully connected layer 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='softmax'))
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=10000,
        decay_rate=0.9)
opt = Adam(learning_rate = lr_schedule)
model.compile(loss='mean_squared_error',  optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 40, 30, 64)        5248      
_________________________________________________________________
batch_normalization (BatchNo (None, 40, 30, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 40, 30, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 20, 15, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 20, 15, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 15, 128)       204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 15, 128)       5

In [4]:
from PIL import Image
from numpy import asarray
import numpy as np
import os
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os
from keras.utils import np_utils

In [5]:
def readImage(filePath):
    img = Image.open(filePath)
    
    size=(30,40)
    #resize image
    out = img.resize(size)

    # asarray() class is used to convert
    # PIL images into NumPy arrays
    numpydata = asarray(out)
    numpydata = np.repeat(numpydata[:, :, np.newaxis], 3, axis=2)
    # <class 'numpy.ndarray'>
    #print(type(numpydata))

    #  shape
    #print(numpydata.shape)
    return numpydata
print(readImage('data/disease/measles (1).png'))

[[[[ 76  84  95]
   [ 76  84  95]
   [ 76  84  95]]

  [[ 15  15  20]
   [ 15  15  20]
   [ 15  15  20]]

  [[  8   7  11]
   [  8   7  11]
   [  8   7  11]]

  ...

  [[ 16  14  25]
   [ 16  14  25]
   [ 16  14  25]]

  [[ 13  13  22]
   [ 13  13  22]
   [ 13  13  22]]

  [[  8   9  13]
   [  8   9  13]
   [  8   9  13]]]


 [[[ 88  96 107]
   [ 88  96 107]
   [ 88  96 107]]

  [[ 17  17  22]
   [ 17  17  22]
   [ 17  17  22]]

  [[  6   5   9]
   [  6   5   9]
   [  6   5   9]]

  ...

  [[ 15  13  23]
   [ 15  13  23]
   [ 15  13  23]]

  [[ 15  13  24]
   [ 15  13  24]
   [ 15  13  24]]

  [[ 10  10  16]
   [ 10  10  16]
   [ 10  10  16]]]


 [[[ 86  91  98]
   [ 86  91  98]
   [ 86  91  98]]

  [[ 14  13  17]
   [ 14  13  17]
   [ 14  13  17]]

  [[  5   4   7]
   [  5   4   7]
   [  5   4   7]]

  ...

  [[ 15  13  23]
   [ 15  13  23]
   [ 15  13  23]]

  [[ 15  13  23]
   [ 15  13  23]
   [ 15  13  23]]

  [[ 12  11  18]
   [ 12  11  18]
   [ 12  11  18]]]


 ...


 [[[157 162 

In [6]:
listFiles = os.listdir('data/disease/')
print(listFiles[:5])

['4164443_down-syndrome-down-syndrome-transparent-background-transparent-png.png', 'main-qimg-4c812ebe5c1bdc504625b6961032fd3b.png', 'measles (1).png', 'measles (10).jpg', 'measles (140).jpg']


In [7]:
#load Benign
listCancer = []
listResults = []
for elem in listFiles:
    img = readImage('data/disease/'+elem)
    listCancer += [img]
    listResults += [np.array([1])]
print(listCancer[:3])
print(img.shape)

[array([[[[243, 243, 243],
         [243, 243, 243],
         [243, 243, 243]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        ...,

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]]],


       [[[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        ...,

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]]],


       

In [8]:
listFiles = os.listdir('data/health/')
print(listFiles[:5])
for elem in listFiles:
    img = readImage('data/health/'+elem)
    listResults += [np.array([0])]
    listCancer += [img]

['1 (1).jpeg', '1 (1).jpg', '1 (1).png', '1 (10).jpeg', '1 (10).jpg']


In [9]:
print(len(listCancer))

200


In [12]:
x_train=[]
y_train=[]
accs=[]
def main(pz):
    
    per = np.random.permutation(len(listCancer))
    ln = int(len(listCancer) * 0.6)
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    positions = []
    #for i in range(ln):
    #    positions += [per[i]]
    val = 1
    while ln > 0:
        val = 1-val
        ln-=1
        for i in range(len(per)):
            if per[i] in positions:
                continue
            if listResults[per[i]] == np.array([val]):
                positions += [per[i]]
                break
    for i in range(len(listCancer)):
        if len(listCancer[i].shape) !=4:
            continue
        if listCancer[i].shape[3] == 4:
            continue
        if i in positions:
            x_train += [listCancer[i]]
            y_train += [listResults[i]]
        else:
            x_test += [listCancer[i]]
            y_test += [listResults[i]]
            
    ln = len(x_test) // 2
    
    x_valid = x_test[ln:]
    y_valid = y_test[ln:]
    
    x_test = x_test[:ln]
    y_test = y_test[:ln]
    
    x_train = np.array(x_train)
    x_train = x_train.reshape(len(x_train), 40, 30, 9)
    y_train = np.array(y_train)
    

    x_test = np.array(x_test)
    x_test = x_test.reshape(len(x_test), 40, 30, 9)
    y_test = np.array(y_test)
    
    x_valid = np.array(x_valid)
    x_valid = x_valid.reshape(len(x_valid), 40, 30, 9)
    y_valid = np.array(y_valid)
    
    model.fit(x_train, y_train,
                epochs = 100,
                batch_size=16,
                validation_data =(x_valid, y_valid),
                shuffle = True)
    scores = model.evaluate(x_test, y_test, verbose = 1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    ret = model.predict(x_test)
    result = []
    pred = []
    for a,b in zip(ret, y_test):
        pred+=[a[0]]
        result+=[b[0]]
    print(pred)
    print(result)
    precision = precision_score(result, pred, average='weighted')
    recall = recall_score(result, pred, average='weighted')
     # calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
    fmeasure = f1_score(result, pred, average='weighted')
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('F1 Score: ', fmeasure)
    #precisions += [precision]
    #recalls += [recall]
    #model.save('saved_models/Sequential'+pz+'.h5')
    return [scores[1], precision, recall, fmeasure]

In [13]:
accs=[]
precisions = []
recalls = []
fmeasures = []
for i in range(10):
    ret = main(i)
    accs+=[ret[0]]
    precisions+=[ret[1]]
    recalls+=[ret[2]]
    fmeasures+=[ret[3]]
print(accs)
print(precisions)
print(recalls)
print(fmeasures)

Epoch 1/100
5/5 [==============================] - 2s 355ms/step - loss: 0.6410 - accuracy: 0.3590 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 2/100
5/5 [==============================] - 2s 343ms/step - loss: 0.6410 - accuracy: 0.3590 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 3/100
5/5 [==============================] - 2s 342ms/step - loss: 0.6410 - accuracy: 0.3590 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 4/100
5/5 [==============================] - 2s 378ms/step - loss: 0.6410 - accuracy: 0.3590 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 5/100
5/5 [==============================] - 2s 370ms/step - loss: 0.6410 - accuracy: 0.3590 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 6/100
5/5 [==============================] - 2s 353ms/step - loss: 0.6410 - accuracy: 0.3590 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 7/100
5/5 [==============================] - 2s 364ms/step - loss: 0.6410 - accuracy: 0.3590 - val_loss: 1.0000 - val_accura

5/5 [==============================] - 2s 345ms/step - loss: 0.6410 - accuracy: 0.3590 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 58/100
5/5 [==============================] - 2s 352ms/step - loss: 0.6410 - accuracy: 0.3590 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 59/100
5/5 [==============================] - 2s 351ms/step - loss: 0.6410 - accuracy: 0.3590 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 60/100
5/5 [==============================] - 2s 340ms/step - loss: 0.6410 - accuracy: 0.3590 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 61/100
5/5 [==============================] - 2s 353ms/step - loss: 0.6410 - accuracy: 0.3590 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 62/100
5/5 [==============================] - 2s 360ms/step - loss: 0.6410 - accuracy: 0.3590 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 63/100
5/5 [==============================] - 2s 395ms/step - loss: 0.6410 - accuracy: 0.3590 - val_loss: 1.0000 - val_accuracy: 0.

c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/100
5/5 [==============================] - 2s 363ms/step - loss: 0.6494 - accuracy: 0.3506 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 2/100
5/5 [==============================] - 2s 362ms/step - loss: 0.6494 - accuracy: 0.3506 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 3/100
5/5 [==============================] - 2s 367ms/step - loss: 0.6494 - accuracy: 0.3506 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 4/100
5/5 [==============================] - 2s 365ms/step - loss: 0.6494 - accuracy: 0.3506 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 5/100
5/5 [==============================] - 2s 365ms/step - loss: 0.6494 - accuracy: 0.3506 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 6/100
5/5 [==============================] - 2s 376ms/step - loss: 0.6494 - accuracy: 0.3506 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 7/100
5/5 [==============================] - 2s 361ms/step - loss: 0.6494 - accuracy: 0.3506 - val_loss: 1.0000 - val_accura

5/5 [==============================] - 2s 362ms/step - loss: 0.6494 - accuracy: 0.3506 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 58/100
5/5 [==============================] - 2s 364ms/step - loss: 0.6494 - accuracy: 0.3506 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 59/100
5/5 [==============================] - 2s 360ms/step - loss: 0.6494 - accuracy: 0.3506 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 60/100
5/5 [==============================] - 2s 363ms/step - loss: 0.6494 - accuracy: 0.3506 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 61/100
5/5 [==============================] - 2s 363ms/step - loss: 0.6494 - accuracy: 0.3506 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 62/100
5/5 [==============================] - 2s 370ms/step - loss: 0.6494 - accuracy: 0.3506 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 63/100
5/5 [==============================] - 2s 379ms/step - loss: 0.6494 - accuracy: 0.3506 - val_loss: 1.0000 - val_accuracy: 0.

...

5/5 [==============================] - 1s 223ms/step - loss: 0.6234 - accuracy: 0.3766 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 58/100
5/5 [==============================] - 1s 225ms/step - loss: 0.6234 - accuracy: 0.3766 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 59/100
5/5 [==============================] - 1s 222ms/step - loss: 0.6234 - accuracy: 0.3766 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 60/100
5/5 [==============================] - 1s 260ms/step - loss: 0.6234 - accuracy: 0.3766 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 61/100
5/5 [==============================] - 1s 266ms/step - loss: 0.6234 - accuracy: 0.3766 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 62/100
5/5 [==============================] - 1s 224ms/step - loss: 0.6234 - accuracy: 0.3766 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 63/100
5/5 [==============================] - 1s 236ms/step - loss: 0.6234 - accuracy: 0.3766 - val_loss: 1.0000 - val_accuracy: 0.

c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5/5 [==============================] - 1s 241ms/step - loss: 0.6575 - accuracy: 0.3425 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 2/100
5/5 [==============================] - 1s 231ms/step - loss: 0.6575 - accuracy: 0.3425 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 3/100
5/5 [==============================] - 1s 217ms/step - loss: 0.6575 - accuracy: 0.3425 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 4/100
5/5 [==============================] - 1s 232ms/step - loss: 0.6575 - accuracy: 0.3425 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 5/100
5/5 [==============================] - 1s 246ms/step - loss: 0.6575 - accuracy: 0.3425 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 6/100
5/5 [==============================] - 1s 249ms/step - loss: 0.6575 - accuracy: 0.3425 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 7/100
5/5 [==============================] - 1s 243ms/step - loss: 0.6575 - accuracy: 0.3425 - val_loss: 1.0000 - val_accuracy: 0.0000e+

5/5 [==============================] - 1s 227ms/step - loss: 0.6575 - accuracy: 0.3425 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 58/100
5/5 [==============================] - 1s 237ms/step - loss: 0.6575 - accuracy: 0.3425 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 59/100
5/5 [==============================] - 1s 235ms/step - loss: 0.6575 - accuracy: 0.3425 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 60/100
5/5 [==============================] - 1s 224ms/step - loss: 0.6575 - accuracy: 0.3425 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 61/100
5/5 [==============================] - 1s 262ms/step - loss: 0.6575 - accuracy: 0.3425 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 62/100
5/5 [==============================] - 1s 254ms/step - loss: 0.6575 - accuracy: 0.3425 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 63/100
5/5 [==============================] - 1s 213ms/step - loss: 0.6575 - accuracy: 0.3425 - val_loss: 1.0000 - val_accuracy: 0.

c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
print(len(accs))
print('accs: ', accs)
print('precisions: ', precisions)
print('recalls: ', recalls)
print('fmeasures: ', fmeasures)

10
accs:  [0.692307710647583, 0.7307692170143127, 0.807692289352417, 0.8214285969734192, 0.8214285969734192, 0.7599999904632568, 0.7407407164573669, 0.7037037014961243, 0.6538461446762085, 0.75]
precisions:  [0.47928994082840237, 0.5340236686390532, 0.6523668639053255, 0.6747448979591837, 0.6747448979591837, 0.5776, 0.5486968449931412, 0.49519890260631, 0.42751479289940825, 0.5625]
recalls:  [0.6923076923076923, 0.7307692307692307, 0.8076923076923077, 0.8214285714285714, 0.8214285714285714, 0.76, 0.7407407407407407, 0.7037037037037037, 0.6538461538461539, 0.75]
fmeasures:  [0.5664335664335665, 0.6170940170940169, 0.7217675941080196, 0.7408963585434174, 0.7408963585434174, 0.6563636363636364, 0.6304176516942475, 0.5813204508856683, 0.5169946332737031, 0.6428571428571429]


In [15]:
import numpy as np
import scipy.stats as st

#define sample data
data = accs

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.7077442123860297, 0.7886391804247918)

In [17]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
x_train = []
y_train = []
x_test = []
y_test = []
positions = []
ln = int(len(listCancer) * 0.8)
per = np.random.permutation(len(listCancer))
for i in range(ln):
    positions += [per[i]]
for i in range(len(listCancer)):
    if len(listCancer[i].shape) !=4:
        continue
    if listCancer[i].shape[3] == 4:
        continue
    if i in positions:
        x_train += [listCancer[i]]
        y_train += [listResults[i]]
    else:
        x_test += [listCancer[i]]
        y_test += [listResults[i]]

ln = len(x_test) // 2

x_valid = x_test[ln:]
y_valid = y_test[ln:]

x_test = x_test[:ln]
y_test = y_test[:ln]

x_train = np.array(x_train)
x_train = x_train.reshape(len(x_train), 40*30*9)
y_train = np.array(y_train)


x_test = np.array(x_test)
x_test = x_test.reshape(len(x_test), 40*30*9)
y_test = np.array(y_test)

x_valid = np.array(x_valid)
x_valid = x_valid.reshape(len(x_valid), 40*30*9)
y_valid = np.array(y_valid)

model.fit(x_train, y_train)

print(model.predict(x_train))
print(y_train)

c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
import pickle

#
# Create your model here (same as above)
#

# Save to file in the current working directory
pkl_filename = "face.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

In [1]:
# pt Sequential

accs=  [0.7857142686843872, 0.5384615659713745, 0.5714285969734192, 0.5384615659713745, 0.7692307829856873, 0.5, 0.3636363744735718, 0.4615384638309479, 0.5555555820465088, 0.8181818127632141]
precisions=  [0.6173469387755102, 0.28994082840236685, 0.32653061224489793, 0.28994082840236685, 0.591715976331361, 0.25, 0.1322314049586777, 0.21301775147928995, 0.30864197530864196, 0.6694214876033059]
recalls=  [0.7857142857142857, 0.5384615384615384, 0.5714285714285714, 0.5384615384615384, 0.7692307692307693, 0.5, 0.36363636363636365, 0.46153846153846156, 0.5555555555555556, 0.8181818181818182]
fmeasures=  [0.6914285714285714, 0.37692307692307697, 0.4155844155844156, 0.37692307692307697, 0.6688963210702342, 0.3333333333333333, 0.19393939393939394, 0.291497975708502, 0.39682539682539686, 0.7363636363636363]

In [16]:
import numpy as np
import scipy.stats as st

#define sample data
data = accs

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

data = precisions

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

data = recalls

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

data = fmeasures

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

(0.7077442123860297, 0.7886391804247918)
(0.5022603416669147, 0.6230758202910869)
(0.7077442172445678, 0.7886391771388266)
(0.5871801572549755, 0.6958281247043916)
